In [1]:
#!/usr/bin/env python3
import sys
import re
import sevenbridges as sbg
import pdb
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import concurrent.futures
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config, error_handlers=[
                sbg.http.error_handlers.rate_limit_sleeper,
                sbg.http.error_handlers.maintenance_sleeper])

In [2]:
project = 'kfdrc-harmonization/sd-bhjxbdqk-06'
manifest = open('/Users/brownm28/Documents/2019-Apr-16_RNAseq_qc/genomic_bam_output.csv')
out_fh = open('/Users/brownm28/Documents/2019-Apr-16_RNAseq_qc/genomic_bam_flagstat_tasks.txt','w')
head = next(manifest)
bam_id_list = []
for line in manifest:
    info = line.split(',')
    bam_id_list.append(api.files.get(id=info[0]))
app_name = project + '/kfdrc-flagstat-qc'
total = len(bam_id_list)
ct = 1
n = 36
for i in range(0, total, n):
    # pdb.set_trace()
    inputs = {}
    bset = i + n
    if bset > total:
        bset = total
    inputs['input_bam'] = bam_id_list[i:bset]
    task_name = 'CBTTC-RNASEQ-GBAM-FLAGSTAT: SET' + str(ct)
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=inputs, run=False)
    task.save()
    out_fh.write(task.name + '\t' + task.id + '\n')
    ct += 1
out_fh.close()

In [3]:
task_file = open('/Users/brownm28/Documents/2019-Apr-16_RNAseq_qc/task_ids.txt')
task_ids = []
i = 0
n = 99
meta_task = []
out = open('/Users/brownm28/Documents/2019-Apr-16_RNAseq_qc/task_basic_info.txt', 'w')
tasks = None
for line in task_file:
    task_ids.append(line.rstrip('\n'))
    if i % n == 0:
        tasks = api.tasks.bulk_get(task_ids)
        meta_task.append(tasks)
        sys.stderr.write('Hit limit, dump and pump!\n')
        task_ids = []
    i += 1
for batch in meta_task:
    for task in batch:
        out.write(task.resource.name + '\t' + task.resource.id + '\n')
tasks = api.tasks.bulk_get(task_ids)
if tasks != meta_task[-1]:
    for task in tasks:
        out.write(task.resource.name + '\t' + task.resource.id + '\n')
out.close()

Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
Hit limit, dump and pump!
